In [ ]:
import cv2
import mediapipe as mp
from ultralytics import YOLO
import numpy as np
import cvzone
import pygame
import telepot
import threading
import os

path = r"D:\Computer Vision\Real-time-Fall-Detection-Using-YOLOV8-Computer-vision--main"
os.chdir(path)

from tracker import Tracker

mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)


pygame.init()
pygame.mixer.music.load("alarm.wav")

bot = telepot.Bot("7234203961:AAFBQOtDtMvNyUYkwqKZ_idXbKcqMH_0JU0")
chat_id = '5045069081'

def send_photo_to_telegram(image_path):
    bot.sendPhoto(chat_id, open(image_path, 'rb'))

    
# YOLO Model
model = YOLO("yolov8n.pt")

# COCO Data
with open("coco.txt", "r") as f:
    classes = f.read().split("\n")

cap = cv2.VideoCapture("fall.mp4")

threshold = 1

# Initialize Tracker
tracker = Tracker()

while True:
    ret, frame = cap.read()
    if not ret:
        print("Error loading video.")
        break

    frame = cv2.resize(frame, (1020, 700))

    # Convert to RGB
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = pose.process(rgb_frame)

    # Pose Detection
    if result.pose_landmarks:
        mp_drawing.draw_landmarks(frame, result.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(0, 255, 255), thickness=2, circle_radius=2),
                                  mp_drawing.DrawingSpec(color=(255, 0, 0), thickness=2, circle_radius=2))

    results = model(frame)[0]

    lis = []
    if results:
        for box in results.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            if int(box.cls[0]) < len(classes):
                name = classes[int(box.cls[0])]
                
                if "person" in name and box.conf[0] > 0.5:
                    lis.append([x1, y1, x2, y2])
                    
                    bbox_id = tracker.update(lis)
                    for bb in bbox_id:
                        x1, y1, x2, y2, idd = bb

                        ww = x2 - x1
                        hh = y2 - y1
                        cvzone.putTextRect(frame, f" Person ID: {idd}", (x1 + 10, y1 - 10), scale=1.5, thickness=2, colorR=(0, 255, 0), colorT=(0, 0, 0))
                        cvzone.cornerRect(frame, (x1, y1, ww, hh), l=20, t=10, rt=2, colorR=(0, 0, 0), colorC=(0, 255, 0))

                    width = x2 - x1
                    height = y2 - y1
                    fall_threshold = width / height
                    print("Fall threshold: ", fall_threshold)
                    
                    if fall_threshold > threshold:
                        cvzone.putTextRect(frame, "Fall Detection !", (x1 + 10, y1 - 30), scale=2, thickness=3, colorR=(0, 255, 0), colorT=(0, 0, 255))
                        cvzone.cornerRect(frame, (x1, y1, width, height), l=20, t=10, rt=2, colorR=(0, 0, 0), colorC=(0, 0, 255))
                        
                        if not pygame.mixer.music.get_busy():
                            pygame.mixer.music.play()
                        
                        image_path = "detected_frame.jpg"
                        cv2.imwrite(image_path, frame)

                        send_photo_to_telegram(image_path)
        
                else:
                    pass

    cv2.imshow("Fall Detection", frame)

    if cv2.waitKey(1) == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()
